# MLflow Models:

The mlflow.models module provides an API for saving machine learning models in “flavors” that can be understood by different downstream tools.

## Sklearn Flavor

The mlflow.sklearn module provides an API for logging and loading scikit-learn models. This module exports scikit-learn models with the following flavors:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
import mlflow 
from mlflow.models.signature import infer_signature
from mlflow_for_ml_dev.experiments.exp_utils import get_or_create_experiment 
from mlflow_for_ml_dev.experiments.exp_utils import get_root_project

In [ ]:
# Create or get the experiment
experiment_name = "sklearn_models"
experiment = get_or_create_experiment(
    experiment_name = experiment_name,
    tags = {
        "proejct_name": "logging_models",
        "topic":"run_management",
        "mlflow.note.content": "This experiment is used to log sklearn models"
    }
)

## Basic Logging

In [ ]:
rfc = RandomForestClassifier()

with mlflow.start_run(run_name="basic_logging", experiment_id=experiment.experiment_id):
    # Log the model
    mlflow.sklearn.log_model(sk_model=rfc, artifact_path = rfc.__class__.__name__)


## Model With Signature

In [ ]:
# load the iris dataset
iris = load_iris(as_frame=True)
x = iris.data
y = iris.target

# infer signature
signature = infer_signature(model_input=x, model_output=y)

In [ ]:
rfc = RandomForestClassifier()

with mlflow.start_run(run_name="logging_with_signature", experiment_id=experiment.experiment_id):
    # Log the model
    mlflow.sklearn.log_model(sk_model=rfc, artifact_path = rfc.__class__.__name__, signature=signature)

## Logging with input example

In [ ]:
# get a sample input
input_example = x.iloc[0:10]
print(input_example)

In [ ]:
rfc = RandomForestClassifier()

with mlflow.start_run(run_name="logging_with_input_example", experiment_id=experiment.experiment_id): 
    # Log the model
    mlflow.sklearn.log_model(
        sk_model = rfc,
        artifact_path = rfc.__class__.__name__,
        signature = signature,
        input_example = input_example
    )

## Logging with code paths 

In [ ]:
# get a sample input
input_example = x.iloc[0:10]
project_path = get_root_project()
rfc = RandomForestClassifier()

with mlflow.start_run(run_name="logging_with_code_paths", experiment_id=experiment.experiment_id):
    # Log the model
    mlflow.sklearn.log_model(
        sk_model=rfc,
        artifact_path = rfc.__class__.__name__,
        signature=signature,
        input_example=input_example,
        code_paths=[(project_path / "mlflow_for_ml_dev").as_posix()])

## Registering the Model

In [ ]:
registered_model_name = "iris_rfc"

# get a sample input
input_example = x.iloc[0:10]
rfc = RandomForestClassifier()

with mlflow.start_run(run_name="logging_and_registering", experiment_id=experiment.experiment_id):

    # Log the model
    mlflow.sklearn.log_model(
        sk_model=rfc,
        artifact_path = rfc.__class__.__name__,
        signature=signature,
        input_example=input_example,
        code_paths=[(project_path / "mlflow_for_ml_dev").as_posix()],
        registered_model_name=registered_model_name
    )

**NOTE:**
----
 While it can be valid to wrap the entire code within the start_run block, this is not recommended. If there as in issue with the training of the model or any other portion of code that is unrelated to MLflow-related actions, an empty or partially-logged run will be created, which will necessitate manual cleanup of the invalid run. It is best to keep the training execution outside of the run context block to ensure that the loggable content (parameters, metrics, artifacts, and the model) are fully materialized prior to logging.